<a href="https://colab.research.google.com/github/dnanper/TEST-langchain/blob/main/test02_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-core>=0.3.47 langchain-openai>=0.3.9 langchain-community==0.3.16

In [2]:
!pip install langsmith

In [19]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchaintest"

In [20]:
import os
from getpass import getpass

os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY") or getpass(
    "Enter OpenRouter API Key: "
)

openrouter_model = "deepseek/deepseek-r1:free"

In [21]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [22]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

In [23]:
prompt_template.input_variables

['context', 'query']

In [24]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [25]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [26]:
prompt_template_2 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [27]:
prompt_template_2.input_variables

['context', 'query']

In [28]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.0,
    api_key = os.getenv("OPENROUTER_API_KEY"),
    base_url = "https://openrouter.ai/api/v1",
    model=openrouter_model
    )

In [29]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template
    | llm
)

In [17]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [18]:
pipeline.invoke({
    "query": query,
    "context": context
    })

AIMessage(content='Aurelio AI is an AI company that develops tooling for AI engineers, with a focus on language AI. They specialize in building AI agents and have expertise in information retrieval. The company is known for open-source frameworks like **Semantic Router** and **Semantic Chunkers**, and they offer an **AI Platform** to help engineers build AI solutions. Additionally, Aurelio AI provides development services to assist organizations in bringing their AI technologies to market. They are also recognized as **LangChain Experts** (as of September 2024) for their work with the LangChain ecosystem.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 343, 'prompt_tokens': 187, 'total_tokens': 530, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1:free', 'system_fingerprint': None, 'id': 'gen-1747890682-WCFTWpupU7PTMfsswRSs', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': No

### Few Shot

In [30]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

In [39]:
sample_examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]


In [32]:
from langchain.prompts import FewShotChatMessagePromptTemplate

In [40]:
fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=sample_examples
)
# here is the formatted prompt
print(fewshot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


In [35]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

In [36]:
prompt_template_2.messages[0].prompt.template = new_system_prompt
prompt_template_2

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nAlways answer in markdown format. When doing so please\nprovide headers, short summaries, follow with bullet\npoints, then conclude.\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [37]:
from IPython.display import display, Markdown

In [43]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "\n## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "\n## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [44]:
fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [45]:
display(Markdown(fewshot_prompt.format()))

Human: Can you explain gravity?
AI: 
## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: 
## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [46]:
# now prompt template use different system prompt, and also add fewshot_prompt
prompt_template_fewshot = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    fewshot_prompt,
    ("human", "{query}"),
])

In [47]:
pipeline_fewshot = (
    {
        "query": lambda x : x["query"],
        "context": lambda x : x["context"]
    }
    | prompt_template_fewshot
    | llm
)

In [51]:
fs_out = pipeline_fewshot.invoke({
    "query": query,
    "context": context
}).content
display(Markdown(fs_out))

## Aurelio AI Overview  
Aurelio AI is an AI company specializing in tools and solutions for AI engineers, with a focus on language-based AI technologies.

### Core Focus Areas  
- **Language AI Development**: Expertise in building AI agents and information retrieval systems.  
- **Open-Source Frameworks**: Created tools like **Semantic Router** (for dynamic decision-making in AI workflows) and **Semantic Chunkers** (for context-aware text processing).  
- **AI Platform**: Provides engineers with tooling to streamline AI development.  
- **Development Services**: Helps organizations commercialize their AI technologies.  

### Recognition  
- Named **LangChain Experts** (September 2024) for their proven track record in delivering solutions using the LangChain ecosystem.  

**To conclude**, Aurelio AI bridges advanced language AI research with practical engineering applications, offering both open-source tools and tailored development services.

### COT

In [52]:
query_cot = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

In [53]:
cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])
cot_pipeline = cot_prompt_template | llm

In [56]:
cot_result = cot_pipeline.invoke({"query": query_cot}).content
display(Markdown(cot_result))

To determine the total number of keystrokes required to type the numbers from 1 to 500, we analyze the problem in three parts:

1. **Single-digit numbers (1–9):**  
   - **Count:** 9 numbers (1 to 9).  
   - **Keystrokes per number:** 1.  
   - **Total:** \(9 \times 1 = 9\) keystrokes.  

2. **Two-digit numbers (10–99):**  
   - **Count:** 90 numbers (10 to 99).  
   - **Keystrokes per number:** 2.  
   - **Total:** \(90 \times 2 = 180\) keystrokes.  

3. **Three-digit numbers (100–500):**  
   - **Count:** 401 numbers (100 to 500, inclusive).  
   - **Keystrokes per number:** 3.  
   - **Total:** \(401 \times 3 = 1,\!203\) keystrokes.  

**Final Calculation:**  
\[
9 \, (\text{single-digit}) + 180 \, (\text{two-digit}) + 1,\!203 \, (\text{three-digit}) = \boxed{1,\!392} \, \text{keystrokes}.
\]

**Answer:**  
1,392 keystrokes are needed to type the numbers from 1 to 500.